In [59]:
from keras.datasets import boston_housing
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import KFold
import numpy as np

In [119]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

mean = np.mean(y_train)
std = np.std(y_train)

y_train = (y_train - mean) / std
y_test = (y_test - mean) / std

print(mean, std)

22.395049504950492 9.199035423364862


In [138]:
batch_size = 8
epochs = 5

In [122]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(13,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 512)               7168      
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_19 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_20 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 513       
Total para

In [135]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mse'])

In [139]:
kf = KFold(n_splits=10)
for train, test in kf.split(x_train, y_train):
    model.fit(x_train[train], y_train[train], 
              batch_size=batch_size, 
              epochs=epochs, 
              verbose=1, 
              validation_data=(x_train[test], y_train[test]))

Train on 363 samples, validate on 41 samples
Epoch 1/5
363/363 [==============================] - 0s 870us/step - loss: 0.2239 - mean_squared_error: 0.2239 - val_loss: 0.1887 - val_mean_squared_error: 0.1887
Epoch 2/5
363/363 [==============================] - 0s 832us/step - loss: 0.2475 - mean_squared_error: 0.2475 - val_loss: 0.2352 - val_mean_squared_error: 0.2352
Epoch 3/5
363/363 [==============================] - 0s 753us/step - loss: 0.2041 - mean_squared_error: 0.2041 - val_loss: 0.1826 - val_mean_squared_error: 0.1826
Epoch 4/5
363/363 [==============================] - 0s 808us/step - loss: 0.1875 - mean_squared_error: 0.1875 - val_loss: 0.1791 - val_mean_squared_error: 0.1791
Epoch 5/5
363/363 [==============================] - 0s 892us/step - loss: 0.1843 - mean_squared_error: 0.1843 - val_loss: 0.2166 - val_mean_squared_error: 0.2166
Train on 363 samples, validate on 41 samples
Epoch 1/5
363/363 [==============================] - 0s 885us/step - loss: 0.2030 - mean_square

364/364 [==============================] - 0s 736us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1348 - val_mean_squared_error: 0.1348
Epoch 4/5
364/364 [==============================] - 0s 834us/step - loss: 0.2163 - mean_squared_error: 0.2163 - val_loss: 0.1123 - val_mean_squared_error: 0.1123
Epoch 5/5
364/364 [==============================] - 0s 744us/step - loss: 0.2095 - mean_squared_error: 0.2095 - val_loss: 0.1149 - val_mean_squared_error: 0.1149


In [140]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3987903583283518
Test accuracy: 0.3987903583283518
